
## Import the necessary libraries

In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from sklearn.metrics import *
from sklearn import preprocessing
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import train_test_split
import re #do regex operations
from wordcloud import WordCloud,STOPWORDS 
from sklearn.preprocessing import LabelEncoder

#Visualize
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud,STOPWORDS
#
# metrics
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import f1_score, precision_score, recall_score

# models
from sklearn.svm import SVC
import time

### loading The Dataset

In [ ]:
#LOADING THE DATASET
df1 = pd.read_csv("/content/fake_job_postings.csv")
df1.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


##Check All informetion about Dataset

In [ ]:
df1.dtypes

job_id                  int64
title                  object
location               object
department             object
salary_range           object
company_profile        object
description            object
requirements           object
benefits               object
telecommuting           int64
has_company_logo        int64
has_questions           int64
employment_type        object
required_experience    object
required_education     object
industry               object
function               object
fraudulent              int64
dtype: object

In [ ]:
df1.shape

(17880, 18)

In [ ]:
# Check for NaN values in the data
df1.isnull().sum().sort_values(ascending = False).loc[df1.dtypes == 'object']


salary_range           15012
department             11547
required_education      8105
benefits                7210
required_experience     7050
function                6455
industry                4903
employment_type         3471
company_profile         3308
requirements            2695
location                 346
description                1
title                      0
dtype: int64

In [ ]:
# Replace NaN values with a blank entry
df1.fillna('', inplace = True)

# Check null Values

In [ ]:
# Check for NaN values in the data
df1.isnull().sum().sort_values(ascending = False).loc[df1.dtypes == 'object']


title                  0
function               0
industry               0
required_education     0
required_experience    0
employment_type        0
benefits               0
requirements           0
description            0
company_profile        0
salary_range           0
department             0
location               0
dtype: int64

##### Demo

In [ ]:
A=df1['description'][0]

In [ ]:
B=df1['requirements'][0]

In [ ]:
A+''+str(print("---"))+B

---


'Food52, a fast-growing, James Beard Award-winning online food community and crowd-sourced and curated recipe hub, is currently interviewing full- and part-time unpaid interns to work in a small team of editors, executives, and developers in its New York City headquarters.Reproducing and/or repackaging existing Food52 content for a number of partner sites, such as Huffington Post, Yahoo, Buzzfeed, and more in their various content management systemsResearching blogs and websites for the Provisions by Food52 Affiliate ProgramAssisting in day-to-day affiliate program support, such as screening affiliates and assisting in any affiliate inquiriesSupporting with PR &amp; Events when neededHelping with office administrative work, such as filing, mailing, and preparing for meetingsWorking with developers to document bugs and suggest improvements to the siteSupporting the marketing and executive staffNoneExperience with content management systems a major plus (any blogging counts!)Familiar wit

In [ ]:
JobPostDF_Final = df1.copy()
JobPostDF_Final.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,,0,1,0,Other,Internship,,,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",,,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,,0,1,0,,,,,,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",,,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [ ]:
JobPostDF_Final['text-corpus'] = JobPostDF_Final['description'] + ' ' + JobPostDF_Final['requirements']
JobPostDF_Final.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent,text-corpus
0,1,Marketing Intern,"US, NY, New York",Marketing,,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,,0,1,0,Other,Internship,,,Marketing,0,"Food52, a fast-growing, James Beard Award-winn..."
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,,Marketing and Advertising,Customer Service,0,Organised - Focused - Vibrant - Awesome!Do you...
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",,,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,,0,1,0,,,,,,0,"Our client, located in Houston, is actively se..."
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0,THE COMPANY: ESRI – Environmental Systems Rese...
4,5,Bill Review Manager,"US, FL, Fort Worth",,,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0,JOB TITLE: Itemization Review ManagerLOCATION:...


In [ ]:
for col in JobPostDF_Final.select_dtypes(include=['object']).drop(columns=['text-corpus']):
    JobPostDF_Final.drop(columns=[col], inplace = True)

In [ ]:
JobPostDF_Final.head()

,job_id,telecommuting,has_company_logo,has_questions,fraudulent,text-corpus
0,1,0,1,0,0,"Food52, a fast-growing, James Beard Award-winn..."
1,2,0,1,0,0,Organised - Focused - Vibrant - Awesome!Do you...
2,3,0,1,0,0,"Our client, located in Houston, is actively se..."
3,4,0,1,0,0,THE COMPANY: ESRI – Environmental Systems Rese...
4,5,0,1,1,0,JOB TITLE: Itemization Review ManagerLOCATION:...


In [ ]:
JobPostDF_Final = JobPostDF_Final.drop(columns=['job_id','telecommuting', 'has_company_logo', 'has_questions'])
JobPostDF_Final.head()

,fraudulent,text-corpus
0,0,"Food52, a fast-growing, James Beard Award-winn..."
1,0,Organised - Focused - Vibrant - Awesome!Do you...
2,0,"Our client, located in Houston, is actively se..."
3,0,THE COMPANY: ESRI – Environmental Systems Rese...
4,0,JOB TITLE: Itemization Review ManagerLOCATION:...


In [ ]:
JobPostDF_Final['text-corpus'][0]

'Food52, a fast-growing, James Beard Award-winning online food community and crowd-sourced and curated recipe hub, is currently interviewing full- and part-time unpaid interns to work in a small team of editors, executives, and developers in its New York City headquarters.Reproducing and/or repackaging existing Food52 content for a number of partner sites, such as Huffington Post, Yahoo, Buzzfeed, and more in their various content management systemsResearching blogs and websites for the Provisions by Food52 Affiliate ProgramAssisting in day-to-day affiliate program support, such as screening affiliates and assisting in any affiliate inquiriesSupporting with PR &amp; Events when neededHelping with office administrative work, such as filing, mailing, and preparing for meetingsWorking with developers to document bugs and suggest improvements to the siteSupporting the marketing and executive staff Experience with content management systems a major plus (any blogging counts!)Familiar with t

Data Cleaning

In [ ]:
import nltk
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk import tokenize
from nltk.stem import WordNetLemmatizer

In [ ]:
def remove(text):
    words = nltk.RegexpTokenizer(r"\w+").tokenize(text)
    
    return words

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# remove stopwords function
stop = set(stopwords.words('english'))
def test(text):
    words = remove(text)
    stripped_phrase = []
    for word in words:
        if word not in stop:
            stripped_phrase.append(word)

    return " ".join(stripped_phrase)

# perform stemming function



In [ ]:
def stem(text):
    stemmer = PorterStemmer()
    stemmed_phrase = []
    words = remove(text)
    for word in words:
        stemmed_phrase.append(stemmer.stem(word))
    
    return " ".join(stemmed_phrase)

# remove stopwords

In [ ]:
JobPostDF_Final['text-corpus'] = JobPostDF_Final['text-corpus'].apply(test)
JobPostDF_Final['text-corpus'][0]

'Food52 fast growing James Beard Award winning online food community crowd sourced curated recipe hub currently interviewing full part time unpaid interns work small team editors executives developers New York City headquarters Reproducing repackaging existing Food52 content number partner sites Huffington Post Yahoo Buzzfeed various content management systemsResearching blogs websites Provisions Food52 Affiliate ProgramAssisting day day affiliate program support screening affiliates assisting affiliate inquiriesSupporting PR amp Events neededHelping office administrative work filing mailing preparing meetingsWorking developers document bugs suggest improvements siteSupporting marketing executive staff Experience content management systems major plus blogging counts Familiar Food52 editorial voice aestheticLoves food appreciates importance home cooking cooking seasonsMeticulous editor perfectionist obsessive attention detail maddened typos broken links delighted finding fixing themChee

In [ ]:
# Perform stemming
JobPostDF_Final['text-corpus'] = JobPostDF_Final['text-corpus'].apply(stem)
JobPostDF_Final['text-corpus'][0]

'food52 fast grow jame beard award win onlin food commun crowd sourc curat recip hub current interview full part time unpaid intern work small team editor execut develop new york citi headquart reproduc repackag exist food52 content number partner site huffington post yahoo buzzfe variou content manag systemsresearch blog websit provis food52 affili programassist day day affili program support screen affili assist affili inquiriessupport pr amp event neededhelp offic administr work file mail prepar meetingswork develop document bug suggest improv sitesupport market execut staff experi content manag system major plu blog count familiar food52 editori voic aestheticlov food appreci import home cook cook seasonsmeticul editor perfectionist obsess attent detail madden typo broken link delight find fix themcheer pressureexcel commun skillsa multi tasker juggler respons big smallinterest engag social media like twitter facebook pinterestlov problem solv collabor drive food52 forwardthink big

#Extract features

In [ ]:
X = JobPostDF_Final['text-corpus']
X.shape

(17880,)

In [ ]:
X.head()

0    food52 fast grow jame beard award win onlin fo...
1    organis focus vibrant awesom do passion custom...
2    our client locat houston activ seek experienc ...
3    the compani esri environment system research i...
4    job titl item review managerloc fort worth tx ...
Name: text-corpus, dtype: object

In [ ]:
Y = JobPostDF_Final['fraudulent']
Y.shape

(17880,)

In [ ]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [ ]:
encoder = LabelEncoder()
train_y = encoder.fit_transform(train_y)
test_y = encoder.fit_transform(test_y)

In [ ]:
train_y

array([0, 0, 0, ..., 1, 0, 0])

# Transform data to count matrix

In [ ]:
MAX = 3000

*feature_extraction*:

1-CountVectorizer 

2-TfidfVectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# tfid = TfidfVectorizer(min_df=0, max_df=1, ngram_range=(1,3))
tfid = TfidfVectorizer(max_features = MAX,use_idf=True)

In [ ]:
# Transform train set
tfid_train = tfid.fit_transform(train_X)
# cv_train = cv.fit_transform(train_X)

# Transform test set
tfid_test = tfid.fit_transform(test_X)
# cv_test = cv.fit_transform(test_X)

print('Tfidf_train : ', tfid_train.shape)
# print('cv_train : ', cv_train.shape)

print('Tfidf_test : ', tfid_test.shape)

Tfidf_train :  (14304, 3000)
Tfidf_test :  (3576, 3000)


#Modelling

Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB
# set the model
nb_model = MultinomialNB()
# nb_model = GaussianNB()

# fit the dataset
nb_tfidf = nb_model.fit(tfid_train, train_y)
print(nb_tfidf)

MultinomialNB()


In [ ]:
nb_tfidf_predict = nb_model.predict(tfid_test)

In [ ]:
nb_tfidf_predict

array([0, 0, 0, ..., 0, 1, 0])

In [ ]:
nb_tfidf_score=accuracy_score(test_y, nb_tfidf_predict)
print("Accuracy  {:.3} %".format(accuracy_score(test_y, nb_tfidf_predict)*100))

Accuracy  85.9 %


In [ ]:
report = classification_report(test_y, nb_tfidf_predict, target_names = ['0', '1'])
print(report)

              precision    recall  f1-score   support

           0       0.95      0.90      0.92      3423
           1       0.01      0.03      0.02       153

    accuracy                           0.86      3576
   macro avg       0.48      0.46      0.47      3576
weighted avg       0.91      0.86      0.89      3576



## SVM 

In [ ]:
svc = SVC(kernel='rbf')
svc.fit(tfid_train, train_y)

SVC()

In [ ]:
predicted = svc.predict(tfid_test)

In [ ]:
print("accuracy score of Support Vector Machine:", accuracy_score(predicted, test_y), "\n")

accuracy score of Support Vector Machine: 0.9572147651006712 



In [ ]:
# creating a confusion matrix 
cm = confusion_matrix(test_y, predicted)
print("confusion matrix of SVM:\n", cm, "\n")


confusion matrix of SVM:
 [[3423    0]
 [ 153    0]] 



In [ ]:
print("classification report:\n", classification_report(test_y, predicted), "\n")

classification report:
               precision    recall  f1-score   support

           0       0.96      1.00      0.98      3423
           1       0.00      0.00      0.00       153

    accuracy                           0.96      3576
   macro avg       0.48      0.50      0.49      3576
weighted avg       0.92      0.96      0.94      3576
 



Using Deep learing 

In [ ]:
import tensorflow
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [ ]:
df =df1[['description','requirements','fraudulent']]
df.head()

,description,requirements,fraudulent
0,"Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,0
1,Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,0
2,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,0
3,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",0
4,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,0


In [ ]:
df.shape

(17880, 3)

In [ ]:
###Drop Nan Values
df=df.dropna()

In [ ]:
df.shape

(17880, 3)

In [ ]:
#Dependent feacture
new_X=df.drop('fraudulent',axis=1)
new_X.head()

,description,requirements
0,"Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...
1,Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...
2,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...
3,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi..."
4,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...


In [ ]:
## Get the Dependent features
new_y=df['fraudulent']
new_y.head()

0    0
1    0
2    0
3    0
4    0
Name: fraudulent, dtype: int64

In [ ]:
new_y.value_counts()

0    17014
1      866
Name: fraudulent, dtype: int64

In [ ]:
voc_size=5000

In [ ]:
message = new_X.copy()

In [ ]:
message['description'][1]

"Organised - Focused - Vibrant - Awesome!Do you have a passion for customer service? Slick typing skills? Maybe Account Management? ...And think administration is cooler than a polar bear on a jetski? Then we need to hear you!\xa0We are the Cloud Video Production Service and opperating on a glodal level. Yeah, it's pretty cool. Serious about\xa0delivering a world class product and excellent customer service.Our rapidly expanding business is looking for a talented Project Manager to manage the successful delivery of video projects, manage client communications and drive the production process. Work with some of the coolest brands on the planet and learn from a global team that are representing NZ is a huge way!We are entering the next growth stage of our business and growing quickly internationally. \xa0Therefore, the position is bursting with opportunity for the right person entering the business at the right time.\xa090 Seconds, the worlds Cloud Video Production Service -\xa0http://90

In [ ]:
message.reset_index(inplace=True)

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(message)):
    review = re.sub('[^a-zA-Z]', ' ', message['description'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
corpus[1]

'organis focus vibrant awesom passion custom servic slick type skill mayb account manag think administr cooler polar bear jetski need hear cloud video product servic opper glodal level yeah pretti cool seriou deliv world class product excel custom servic rapidli expand busi look talent project manag manag success deliveri video project manag client commun drive product process work coolest brand planet learn global team repres nz huge way enter next growth stage busi grow quickli intern therefor posit burst opportun right person enter busi right time second world cloud video product servic http url fbe afac cd c f b eef e e f ca dd second world cloud video product servic enabl brand agenc get high qualiti onlin video content shot produc anywher world fast afford manag seamlessli cloud purchas publish second remov hassl cost risk speed issu work regular video product compani manag everi aspect video project beauti onlin experi grow network rate video profession countri dedic product suc

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr[1]

[2632,
 2550,
 1616,
 982,
 997,
 305,
 1975,
 2098,
 2511,
 3111,
 4818,
 4238,
 4764,
 1693,
 4229,
 2720,
 4871,
 4018,
 914,
 1855,
 2387,
 480,
 4622,
 4534,
 1975,
 898,
 1859,
 2575,
 3850,
 1195,
 4260,
 785,
 504,
 784,
 4225,
 4534,
 1123,
 305,
 1975,
 3206,
 531,
 1592,
 522,
 4029,
 4792,
 4764,
 4764,
 222,
 178,
 4622,
 4792,
 4764,
 3415,
 1164,
 1471,
 4534,
 260,
 3379,
 3040,
 551,
 2002,
 2264,
 1967,
 4401,
 4471,
 3935,
 2312,
 4397,
 4412,
 2879,
 4232,
 4135,
 1592,
 3901,
 3298,
 1292,
 584,
 4480,
 2813,
 4873,
 2900,
 3898,
 4412,
 1592,
 2900,
 62,
 2412,
 784,
 480,
 4622,
 4534,
 1975,
 148,
 2721,
 1836,
 3412,
 1494,
 2070,
 3442,
 2418,
 882,
 2111,
 2111,
 3442,
 3773,
 2875,
 2412,
 784,
 480,
 4622,
 4534,
 1975,
 3771,
 551,
 324,
 3293,
 1497,
 95,
 294,
 4622,
 4051,
 4356,
 1304,
 2640,
 784,
 2030,
 4703,
 4764,
 2805,
 480,
 1975,
 4524,
 2412,
 2152,
 1790,
 1628,
 4354,
 2495,
 1597,
 3379,
 1330,
 4622,
 4534,
 3758,
 4764,
 2803,
 2400,
 46

In [ ]:
sent_length=40
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[1477 4051 4764 ...  485  479  744]
 [ 222 3223 4887 ... 2527  832 4645]
 [   0    0    0 ... 1780 3492 3411]
 ...
 [1238 3566 3435 ... 3187 2223 2197]
 [ 763  791 3583 ... 3935 1292 4792]
 [2068  125 2749 ... 3215  197  935]]


In [ ]:
a=embedded_docs[0]
len(a)

40

In [ ]:
## Creating model
embedding_vector_features=50
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100))) ##Just add bidirectional!!, except it would just behave as normal LSTM Model
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 40, 50)            250000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              120800    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 371,001
Trainable params: 371,001
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
len(embedded_docs),new_y.shape

(17880, (17880,))

In [ ]:
X_final=np.array(embedded_docs)
y_final=np.array(new_y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.25, random_state=32)

In [ ]:
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=12,batch_size=64)

Epoch 1/12
210/210 [==============================] - 27s 103ms/step - loss: 0.1811 - accuracy: 0.9565 - val_loss: 0.1184 - val_accuracy: 0.9653
Epoch 2/12
210/210 [==============================] - 20s 97ms/step - loss: 0.0891 - accuracy: 0.9728 - val_loss: 0.1055 - val_accuracy: 0.9711
Epoch 3/12
210/210 [==============================] - 21s 99ms/step - loss: 0.0568 - accuracy: 0.9822 - val_loss: 0.1078 - val_accuracy: 0.9738
Epoch 4/12
210/210 [==============================] - 21s 100ms/step - loss: 0.0354 - accuracy: 0.9887 - val_loss: 0.1267 - val_accuracy: 0.9734
Epoch 5/12
210/210 [==============================] - 21s 102ms/step - loss: 0.0247 - accuracy: 0.9928 - val_loss: 0.1174 - val_accuracy: 0.9745
Epoch 6/12
210/210 [==============================] - 22s 105ms/step - loss: 0.0130 - accuracy: 0.9963 - val_loss: 0.1506 - val_accuracy: 0.9687
Epoch 7/12
210/210 [==============================] - 20s 98ms/step - loss: 0.0078 - accuracy: 0.9982 - val_loss: 0.1687 - val_accur

In [ ]:
y_pred=model1.predict(X_test)
y_pred

array([[2.4808756e-07],
       [8.1939943e-05],
       [1.3914766e-07],
       ...,
       [9.9997866e-01],
       [2.1594589e-05],
       [1.8661719e-07]], dtype=float32)

In [ ]:
y_pred = [1 if x >=0.5 else 0 for x in y_pred]
y_pred[5:20]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]

In [ ]:
confusion_matrix(y_test,y_pred)

array([[4226,   21],
       [  89,  134]])

In [ ]:
accuracy_score(y_test,y_pred)
print("Accuracy  {:.3} %".format(accuracy_score(y_test, y_pred)*100))

Accuracy  97.5 %


In [ ]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      4247
           1       0.86      0.60      0.71       223

    accuracy                           0.98      4470
   macro avg       0.92      0.80      0.85      4470
weighted avg       0.97      0.98      0.97      4470



In [ ]:
#Accuracy score for Naive Bayes features

nb_tfidf_score=accuracy_score(test_y, nb_tfidf_predict)
print("Naive Bayes Accuracy  {:.3} %".format(accuracy_score(test_y, nb_tfidf_predict)*100))

Naive Bayes Accuracy  85.9 %


In [ ]:
print("Support Vector Machine Accuracy  {:.4} %".format(accuracy_score(predicted, test_y)*100))

Support Vector Machine Accuracy  95.72 %


In [ ]:
print("BiLSTM Accuracy  {:.3} %".format(accuracy_score(y_test, y_pred)*100))
print(classification_report(y_test,y_pred))

BiLSTM Accuracy  97.5 %
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      4247
           1       0.86      0.60      0.71       223

    accuracy                           0.98      4470
   macro avg       0.92      0.80      0.85      4470
weighted avg       0.97      0.98      0.97      4470



## Observetion :
## Using kernel_initializer,  kernel_regularizer(L1 and L2) will get more accuracy

1-he_uniform()
he_uniform draws samples in uniform distribution with -inf to +inf as range.


2-glorot_uniform()
glorot_uniform draws samples in uniform distribution with stddev = sqrt(2 / (fan_in + fan_out)) fan_in is num of units in weight tensor and fan_out is num of output units.
            

